In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
print(os.getcwd())

/Users/snehitha/taxi_project/notebooks


In [3]:
import sys
import os

# Move up one level and add "src" directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

# Now import the module
import taxi_project.config as config

In [4]:
from taxi_project.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from taxi_project.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-03-05 09:22:27,596 INFO: Initializing external client
2025-03-05 09:22:27,597 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 09:22:28,582 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691
Fetching data from 2025-02-04 14:22:27.595162+00:00 to 2025-03-05 13:22:27.595162+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.73s) 


In [5]:
from taxi_project.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 09:22:38,511 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 09:22:38,514 INFO: Initializing external client
2025-03-05 09:22:38,514 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 09:22:39,151 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691


In [6]:
from taxi_project.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 09:22:42,430 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 09:22:42,433 INFO: Initializing external client
2025-03-05 09:22:42,434 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 09:22:43,094 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691


In [7]:
from taxi_project.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [8]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,255,3.0,2025-03-05 15:00:00+00:00
1,112,0.0,2025-03-05 15:00:00+00:00
2,218,0.0,2025-03-05 15:00:00+00:00
3,237,388.0,2025-03-05 15:00:00+00:00
4,57,0.0,2025-03-05 15:00:00+00:00
...,...,...,...
246,108,0.0,2025-03-05 15:00:00+00:00
247,71,0.0,2025-03-05 15:00:00+00:00
248,8,0.0,2025-03-05 15:00:00+00:00
249,168,1.0,2025-03-05 15:00:00+00:00


In [9]:
from taxi_project.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-05 09:22:46,127 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 09:22:46,130 INFO: Initializing external client
2025-03-05 09:22:46,130 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 09:22:46,887 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691


In [10]:
predictions = predictions.rename(columns={"predicted_demand": "prediction"})


In [11]:
predictions = predictions.rename(columns={"prediction": "predicted_demand"})


In [12]:
print(predictions.dtypes)
print(predictions.head())


pickup_location_id                      int32
predicted_demand                      float64
pickup_hour           datetime64[us, Etc/UTC]
dtype: object
   pickup_location_id  predicted_demand               pickup_hour
0                 255               3.0 2025-03-05 15:00:00+00:00
1                 112               0.0 2025-03-05 15:00:00+00:00
2                 218               0.0 2025-03-05 15:00:00+00:00
3                 237             388.0 2025-03-05 15:00:00+00:00
4                  57               0.0 2025-03-05 15:00:00+00:00


In [13]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - prediction (type: 'float') is missing from input dataframe.
 - predicted_demand (type: 'double') does not exist in feature group.
Note that feature (or column) names are case insensitive and spaces are automatically replaced with underscores.